# Waveform
![status](https://img.shields.io/badge/status-in%20progress-orange)

<div style="text-align: center;">
    <img src="./assets/230524_waveform.png" alt="waveform preview" width="450"/>
</div>

## Summary

This waveform workflow will demonstrate oscilloscope-style display of large-scale action potential waveform snippets, scale bar, and grouping by source. 

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [1]:
import panel as pn; pn.extension()
import datashader as ds
import numpy as np
import holoviews as hv; hv.extension('bokeh')
import hvplot.pandas
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
from neurodatagen.ephys import load_waveform_templates, create_noisy_waveforms

## Generate waveform data

The function `create_noisy_waveforms` will add noise to templates loaded with `load_waveform_templates` to generate any number of spikes.

### Load the templates and visualize one of them

In [4]:
i_template = 100
noise_std_percent = 50
n_spikes = 1

wf = load_waveform_templates()
template_wf = wf.iloc[i_template]
noisy_wf = create_noisy_waveforms(template_wf.values, noise_std_percent, n_spikes)

template_wf.hvplot(label='template') * hv.Curve(np.squeeze(noisy_wf), 'time', 'amplitude', label='noise-added')

:Overlay
   .Curve.Template                 :Curve   [index]   (FSH003_009B)
   .Curve.Noise_hyphen_minus_added :Curve   [time]   (amplitude)

## Bonus: Create an app to explore waveform generation parameters

In [5]:
c1_picker = pn.widgets.ColorPicker(name='color', value='#037bfc', width=60)
c2_picker = pn.widgets.ColorPicker(name='color', value='#d13030', width=60)
u1_slider = pn.widgets.IntSlider(name='unit 1', start=0, end=wf.shape[1], value=100, width=150)
u2_slider = pn.widgets.IntSlider(name='unit 2', start=0, end=wf.shape[1], value=55, width=150)
numspk1_slider = pn.widgets.IntSlider(name='n spikes', start=0, end=10000, value=1000, width=150)
numspk2_slider = pn.widgets.IntSlider(name='n spikes', start=0, end=10000, value=1000, width=150)
perc_std_slider1 = pn.widgets.IntSlider(name='noise std %', start=0, end=300, value=50, width=150)
perc_std_slider2 = pn.widgets.IntSlider(name='noise std %', start=0, end=300, value=50, width=150)


def plot_overlay(wfs, c1, c2):
    return datashade(hv.NdOverlay(wfs, kdims='unit'), color_key={1:c1, 2:c2},
                     aggregator=ds.by('unit', ds.count())).opts(
        tools=['hover'], width=600, height=600, xlabel='time', ylabel='amplitude', 
        xlim=(20,100))

def waveform_dash(c1, c2, u1, u2, numspk1, numspk2, noise1, noise2):
    spike_waveform1 = wf.iloc[u1].values
    spike_waveform2 = wf.iloc[u2].values

    noisy_spike_waveforms1 = create_noisy_waveforms(spike_waveform1, num_spikes = numspk1, noise_std_percent=noise1)
    noisy_spike_waveforms2 = create_noisy_waveforms(spike_waveform2, num_spikes = numspk2, noise_std_percent=noise2)

    wfs = {}
    time = np.arange(noisy_spike_waveforms1.shape[1])
    wfs[1] = hv.Path((time, noisy_spike_waveforms1.T))
    wfs[2] = hv.Path((time, noisy_spike_waveforms2.T))
    plot = plot_overlay(wfs, c1, c2) 
    return plot

bound = pn.bind(waveform_dash, c1_picker, c2_picker, u1_slider, u2_slider, 
               numspk1_slider, numspk2_slider, perc_std_slider1, perc_std_slider2)
pn.Row(bound,
       pn.Column(u1_slider, numspk1_slider, perc_std_slider1, c1_picker, pn.Spacer(height=50), 
              u2_slider, numspk2_slider,  perc_std_slider2, c2_picker))

Row
    [0] ParamFunction(function, _pane=HoloViews)
    [1] Column
        [0] IntSlider(end=150, name='unit 1', value=100, width=150)
        [1] IntSlider(end=10000, name='n spikes', value=1000, width=150)
        [2] IntSlider(end=300, name='noise std %', value=50, width=150)
        [3] ColorPicker(name='color', value='#037bfc', width=60)
        [4] Spacer(height=50)
        [5] IntSlider(end=150, name='unit 2', value=55, width=150)
        [6] IntSlider(end=10000, name='n spikes', value=1000, width=150)
        [7] IntSlider(end=300, name='noise std %', value=50, width=150)
        [8] ColorPicker(name='color', value='#d13030', width=60)

## Plot waveform viewer on generated data

## Plot real data